In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
import transformers
import math
import statistics
from tqdm import tqdm
import torch.nn.functional as F
import torch
import os
import numpy as np
import pandas as pd
import seaborn as sns
from transformers import pipeline
from torch.utils.data.dataloader import default_collate
from transformers import GPTNeoForCausalLM, GPT2Tokenizer
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
from torchvision.models import resnet101
from torchvision.models import resnet50
from sklearn.ensemble import IsolationForest
from torch.utils.data import DataLoader , TensorDataset

In [ ]:
# !pwd

/content


In [ ]:
# !pip install av


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.8/33.8 MB 52.6 MB/s eta 0:00:00


In [2]:
import torch
import av
import torchvision
from torch import nn, optim
from torchvision import transforms
from torchvision.datasets import UCF101

In [4]:
frames_per_clip = 5
step_between_clips = 1
batch_size = 128
path='./ucfTrainTestlist'

In [ ]:
# !pwd

/content


In [ ]:
# !unrar x '/content/drive/My Drive/Video/UCF101.rar'

In [13]:
ucf_data_dir = "./UCF101/UCF-101/"
ucf_label_dir = path
frames_per_clip = 5
step_between_clips = 1
batch_size = 128

In [7]:
tfs = transforms.Compose([
            # TODO: this should be done by a video-level transfrom when PyTorch provides transforms.ToTensor() for video
            # scale in [0, 1] of type float
            transforms.Lambda(lambda x: x / 255.),
            # reshape into (T, C, H, W) for easier convolutions
            transforms.Lambda(lambda x: x.permute(0, 3, 1, 2)),
            # rescale to the most common size
            transforms.Lambda(lambda x: nn.functional.interpolate(x, (240, 320))),
])

In [8]:
def custom_collate(batch):
    filtered_batch = []
    for video, _, label in batch:
        filtered_batch.append((video, label))
    return torch.utils.data.dataloader.default_collate(filtered_batch)

In [17]:
# load train test datasets
train_dataset = UCF101(ucf_data_dir, ucf_label_dir, frames_per_clip=frames_per_clip,
                       step_between_clips=step_between_clips, train=True, transform=tfs)

test_dataset = UCF101(ucf_data_dir, ucf_label_dir, frames_per_clip=frames_per_clip,
                      step_between_clips=step_between_clips, train=False, transform=tfs)


100%|██████████| 833/833 [03:03<00:00,  4.54it/s]


In [18]:
### If you want to use a part of the dataset (for small runtime)
train_set, test_set_org = torch.utils.data.random_split(train_dataset,
                                                   [100000, 1647933])
test_set, test_set2 = torch.utils.data.random_split(test_dataset,
                                                   [25000,657084])

In [19]:
# create train test test loader (allowing batches and other extras)

train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True,
                                           collate_fn=custom_collate)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=True,
                                          collate_fn=custom_collate)

In [20]:
#batch_size_detector=len(train_set)
batch_size_detector=128
outlier_detector = DataLoader(train_set,  batch_size=batch_size_detector, shuffle=True, collate_fn=custom_collate)
#batch_size_detector=len(train_set)
batch_size_detector2=128
outlier_detector2 = DataLoader(train_set,  batch_size=batch_size_detector2, shuffle=True, collate_fn=custom_collate)

In [21]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # use gpu if available
# all_preds=[]
# for batch, (X, Y) in enumerate(tqdm(outlier_detector)):

#     X, Y = X.to(device=device), Y.to(device)
#     print("Dataset Loaded!")
#     X_0_1=X[:,0, 0, :,:]
#     X_0_1 = X_0_1.reshape(X.shape[0], X_0_1.shape[1]*X_0_1.shape[2])/255.0

#     #print(X[0][0])
#     val_out=5.00/100.00
#     isf = IsolationForest(contamination=val_out, n_jobs=-1)
#     X_0_1=X_0_1.cpu().detach().numpy()
#     _ = isf.fit(X_0_1)

#     # Predictions
#     preds = isf.predict(X_0_1)
#     pred_list=preds.tolist()
#     all_preds.append(preds)
#     print(pred_list)
#     counter=0
#     for i in enumerate(pred_list):
#       if (i[1]!=-1):
#         counter=counter+1
#         #non_outX=X[X[0]!=i[0]]
# for batch, (X, Y) in enumerate(tqdm(outlier_detector2)):

#     X, Y = X.to(device=device), Y.to(device)
#     #arr = np.array(all_preds)
#     non_outX=X[all_preds!=-1]
#     print(counter, non_outX.shape)


  0%|          | 0/782 [00:00<?, ?it/s]c:\Users\Cameron Keith\anaconda3\envs\base2\Lib\site-packages\torchvision\io\video.py:161: UserWarning: The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.
  warnings.warn("The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.")


Dataset Loaded!


  0%|          | 1/782 [00:06<1:18:31,  6.03s/it]

[1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, -1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


c:\Users\Cameron Keith\anaconda3\envs\base2\Lib\site-packages\torchvision\io\video.py:161: UserWarning: The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.
  warnings.warn("The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.")
  0%|          | 2/782 [00:10<1:07:58,  5.23s/it]

...

Dataset Loaded!
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


c:\Users\Cameron Keith\anaconda3\envs\base2\Lib\site-packages\torchvision\io\video.py:161: UserWarning: The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.
  warnings.warn("The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.")
  7%|▋         | 55/782 [04:31<59:43,  4.93s/it]

Dataset Loaded!
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, -1]


c:\Users\Cameron Keith\anaconda3\envs\base2\Lib\site-packages\torchvision\io\video.py:161: UserWarning: The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.
  warnings.warn("The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.")
  7%|▋         | 55/782 [04:35<1:00:35,  5.00s/it]


KeyboardInterrupt: 

In [22]:
def outlier_removal(dataloader=outlier_detector, percentile=5):
  all_preds=[]
  for batch, (X, Y) in enumerate(tqdm(outlier_detector)):

      X, Y = X.to(device=device), Y.to(device)
      print("Dataset Loaded!")
      X_0_1=X[:,0, 0, :,:]
      X_0_1 = X_0_1.reshape(X.shape[0], X_0_1.shape[1]*X_0_1.shape[2])/255.0

      #print(X[0][0])
      val_out=5.00/100.00
      isf = IsolationForest(contamination=val_out, n_jobs=-1)
      X_0_1=X_0_1.cpu().detach().numpy()
      _ = isf.fit(X_0_1)

      # Predictions
      preds = isf.predict(X_0_1)
      pred_list=preds.tolist()
      all_preds.append(preds)
      print(pred_list)
      counter=0
      for i in enumerate(pred_list):
        if (i[1]!=-1):
          counter=counter+1
          #non_outX=X[X[0]!=i[0]]
  for batch, (X, Y) in enumerate(tqdm(outlier_detector2)):
      X, Y = X.to(device=device), Y.to(device)
      #arr = np.array(all_preds)
      non_outX=X[all_preds!=-1]
      print(counter, non_outX.shape)
      outl=X[preds==-1]
      non_outl=X[all_preds!=-1]
      new_X=non_outl
      print(counter, non_outl.shape)
      #new_X[batch*batch_size_train + len(non_outl)-1]=new_X
      print(f'Batch {batch} >> {len(outl)} Outliers {len(non_outl)} Non-Outliers detected with contamination {val_out}')
      outl_lab = Y[all_preds==-1]
      nonoutl_lab = Y[all_preds!=-1]
      new_Y=nonoutl_lab
      print(new_Y.shape)
      #new_Y[batch*batch_size_train + len(non_outl)-1]=new_Y
      dataset = TensorDataset(new_X , new_Y)

      return dataset

In [23]:
## percentage of samples to remove
psi=5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # use gpu if available
clean_dataset=outlier_removal(dataloader=outlier_detector, percentile=psi)
print(f"Total number of train samples after outlier removal: {len(clean_dataset)}")

train_loader = torch.utils.data.DataLoader(clean_dataset, batch_size=batch_size, shuffle=True,
                                           collate_fn=custom_collate)

  0%|          | 1/782 [00:04<1:04:30,  4.96s/it]

Dataset Loaded!
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, -1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


c:\Users\Cameron Keith\anaconda3\envs\base2\Lib\site-packages\torchvision\io\video.py:161: UserWarning: The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.
  warnings.warn("The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.")
  0%|          | 2/782 [00:09<1:04:48,  4.98s/it]

Dataset Loaded!
[1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


c:\Users\Cameron Keith\anaconda3\envs\base2\Lib\site-packages\torchvision\io\video.py:161: UserWarning: The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.
  warnings.warn("The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.")
  0%|          | 3/782 [00:14<1:03:54,  4.92s/it]

...

Dataset Loaded!
[1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


  0%|          | 0/782 [00:00<?, ?it/s]c:\Users\Cameron Keith\anaconda3\envs\base2\Lib\site-packages\torchvision\io\video.py:161: UserWarning: The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.
  warnings.warn("The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.")


30 torch.Size([1, 128, 5, 3, 240, 320])


  0%|          | 0/782 [00:05<?, ?it/s]


IndexError: The shape of the mask [32] at index 0 does not match the shape of the indexed tensor [128, 5, 3, 240, 320] at index 0

In [ ]:
def custom_collate2(batch):
    filtered_batch = []
    for video, label in batch:
        filtered_batch.append((video, label))
    return torch.utils.data.dataloader.default_collate(filtered_batch)

In [ ]:
train_loader = torch.utils.data.DataLoader(clean_dataset, batch_size=batch_size, shuffle=True,
                                           collate_fn=custom_collate2)

In [ ]:
def target_train(train_loader, target_model, optimiser):
   target_model.train()
   size = len(train_loader.dataset)
   correct = 0
   total_loss=[]
   total_sample=0
   for batch, (X, Y) in enumerate(tqdm(train_loader)):
       X, Y = X.to(device=device), Y.to(device=device)
       #print(X.shape, Y.shape)
       X_0_1=X[:,0, 0, :,:]
       X_0_1 = X_0_1.reshape(X.shape[0], X_0_1.shape[1]*X_0_1.shape[2])/255.0
       #print(X[0][0])
       val_out=5.00/100.00
       isf = IsolationForest(contamination=val_out, n_jobs=-1)
       X_0_1=X_0_1.cpu().detach().numpy()
       _ = isf.fit(X_0_1)


       # Predictions
       preds = isf.predict(X_0_1)
       non_outl=X[preds!=-1]
       new_X=non_outl
       print(non_outl.shape)
       #new_X[batch*batch_size_train + len(non_outl)-1]=new_X
       print(f'Batch {batch} >> {len(non_outl)} Non-Outliers detected with contamination {val_out}')
       outl_lab = Y[preds==-1]
       nonoutl_lab = Y[preds!=-1]
       new_Y=nonoutl_lab
       total_sample=total_sample+new_X.shape[0]
       optimiser.zero_grad()
       pred = target_model(new_X)
       #print(pred, Y)
       #pred = pred.flatten()
       loss = cost(pred, new_Y)
       loss.backward()
       optimiser.step()
       _, output = torch.max(pred, 1)
       #print(output, Y)
       correct+= (output == new_Y).sum().item()
       total_loss.append(loss.item())
       #batch_count+=batch
       #correct += (pred.argmax(1)==Y).type(torch.float).sum().item()


   #print(correct, size)
   correct /= size
   loss= sum(total_loss)/(batch+1)
   print("Total Sample: ", total_sample)
   result_train=100*correct
   torch.save(target_model, '/content/drive/My Drive/Video/50k_model.pt')


   print(f'\nTraining Performance:\nacc: {(100*correct):>0.1f}%, avg loss: {loss:>8f}\n')
  
   return loss, result_train



In [ ]:
class HLSTMCNN(nn.Module):
    def __init__(self, num_classes=101):
        super(HLSTMCNN, self).__init__()
        self.resnet = resnet50(pretrained=True)
        self.resnet.fc = nn.Sequential(nn.Linear(self.resnet.fc.in_features, 300))
        self.lstm = nn.LSTM(input_size=300, hidden_size=256, num_layers=3)
        self.fc1 = nn.Linear(256, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x_3d):
        hidden = None

        # Iterate over each frame of a video in a video of batch * frames * channels * height * width
        for t in range(x_3d.size(1)):
            with torch.no_grad():
                x = self.resnet(x_3d[:, t])
            # Pass latent representation of frame through lstm and update hidden state
            out, hidden = self.lstm(x.unsqueeze(0), hidden)

        # Get the last hidden state (hidden is a tuple with both hidden and cell state in it)
        x = self.fc1(hidden[0][-1])
        x = F.relu(x)
        x = self.fc2(x)

        return x

In [ ]:
device = torch.device("cuda")  # use gpu if available
target_model = HLSTMCNN().to(device=device)
optimiser=torch.optim.SGD(target_model.parameters(),lr=0.01,momentum=0.9)
cost = torch.nn.CrossEntropyLoss()


In [ ]:
def target_utility(test_loader, target_model, batch_size = batch_size):
    size = len(test_loader.dataset)
    target_model.eval()
    test_loss, correct = 0, 0
    correct = 0
    total=0
    counter_a=0
    #with torch.no_grad():
    for batch, (X, Y) in enumerate(tqdm(test_loader)):
        X, Y = X.to(device=device), Y.to(device=device)
        X.requires_grad = True
        pred = target_model(X)
        #print("Y is: ", Y)
        counter_a=counter_a+1
        #test_loss += cost(pred, Y).item()
        total += Y.size(0)
        # Forward pass the data through the model
        _, output_res = torch.max(pred, -1)
        #print("pred is: ",output_res)
        correct += ((output_res) == Y).sum().item()


    # Calculate final accuracy
    final_acc = correct/float(total)
    print(f"Target Model Accuracy = {correct} / {total} = {final_acc}")

    # Return the accuracy
    return final_acc



In [ ]:
import smtplib
from email.mime.text import MIMEText

def send_email(subject, message):
    sender_email = 'cameronkeithgolf@gmail.com'
    receiver_email = 'cameronkeithgolf@gmail.com'
    smtp_server = 'smtp.gmail.com'
    smtp_port = 587
    smtp_username = 'cameronkeithgolf@gmail.com'
    smtp_password = 'lkhauhjcqytyrarr'

    msg = MIMEText(message)
    msg['From'] = sender_email
    msg['Subject'] = subject
    msg['To'] = receiver_email

    server = smtplib.SMTP(smtp_server, smtp_port)
    server.starttls()
    server.login(smtp_username, smtp_password)
    server.sendmail(sender_email, receiver_email, msg.as_string())
    server.quit()


In [ ]:
target_epochs=15
loss_train_tr, loss_test_tr=[],[]
for t in tqdm(range(target_epochs)):
    print(f'Epoch {t+1}\n-------------------------------')
    print("+++++++++Target Training Starting+++++++++")
    tr_loss, result_train=target_train(train_loader, target_model, optimiser)
    loss_train_tr.append(tr_loss)
    # print('Test Accuracy: ', target_utility(test_loader, target_model, batch_size = batch_size))
    if t % 10 == 0: 
        send_email(
        f'CNN Training update after {t+1} epochs',
        f'''
            Train loss: {tr_loss}
            Train accuracy: {result_train}
            Test accuracy: {target_utility(test_loader, target_model, batch_size=batch_size)}
            '''
        )
